In [2]:
%%time

import pandas as pd

from math import isnan

exchange = 'NSE'

#... build the symbols
#______________________

tp = pd.read_html('https://www.tradeplusonline.com/Equity-Futures-Margin-Calculator.aspx')

df_tp = tp[1][2:].iloc[:, :-1]
df_tp = df_tp.iloc[:, [0,1,5]]
df_tp.columns=['nseSymbol', 'lot', 'margin']

cols = df_tp.columns.drop('nseSymbol')
df_tp[cols] = df_tp[cols].apply(pd.to_numeric, errors='coerce') # convert lot and margin to numeric

df_slm = df_tp.copy()

# Truncate to 9 characters for ibSymbol
df_slm['ibSymbol'] = df_slm.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_slm.ibSymbol = df_slm.ibSymbol.replace(ntoi)

# separate indexes and equities, eliminate discards from df_slm
indexes = ['NIFTY50', 'BANKNIFTY']
discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
equities = sorted([s for s in df_slm.ibSymbol if s not in indexes+discards])

symbols = equities+indexes
exchange = 'NSE'

from ib_insync import *
util.startLoop()
ib = IB()

with ib.connect('127.0.0.1', 3000, clientId=13): # For nse - Kavi:
    cs = [Stock(s, exchange) if s in equities else Index(s, exchange) for s in symbols]
    qcs = ib.qualifyContracts(*cs) # qualified underlyings

Started to throttle requests
Stopped to throttle requests


Wall time: 17.9 s


In [6]:
with ib.connect('127.0.0.1', 3000, clientId=13): # For nse - Kavi:
    mdata = []
    for i in range(0, len(qcs), 100):
        for q in qcs[i:i+100]:
            mdata.append(ib.reqMktData(contract=q, genericTickList='456', snapshot=False, regulatorySnapshot=False))
    

Started to throttle requests


In [8]:
mdata

[Ticker(contract=Stock(conId=44652144, symbol='ACC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ACC', tradingClass='ACC'), ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[]),
 Ticker(contract=Stock(conId=56986798, symbol='ADANIENT', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ADANIENT', tradingClass='ADANIENT'), ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[]),
 Ticker(contract=Stock(conId=79403918, symbol='ADANIPORT', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ADANIPORTS', tradingClass='ADANIPORT'), ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[]),
 Ticker(contract=Stock(conId=70299711, symbol='ADANIPOWE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ADANIPOWER', tradingClass='ADANIPOWE'), ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[]),
 Ticker(contract=Stock(conId=187395649, symbol='AJANTPHAR', exchange='NSE', primaryExchange='NSE', 

In [ ]:
[ib.reqTickers(q) for i in range(0, len(qcs), 50) for q in qcs[i: i+50]]
            

In [ ]:
qs

In [ ]:
    
    mdata = [ib.reqMktData(contract=q, genericTickList='456', snapshot=False, regulatorySnapshot=False) for i in range(0, len(qcs), 50) for q in qcs[i:i+50]]
    ib.sleep(10)